Gradio APP

In [ ]:
# Install the required libraries
!pip install transformers gradio Pillow requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00


In [ ]:
import os
import requests
from transformers import MarianMTModel, MarianTokenizer, AutoModelForCausalLM, AutoTokenizer
from PIL import Image, ImageDraw
import io
import gradio as gr
import torch

# Detect if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the MarianMT model and tokenizer for translation (Tamil to English)
model_name = "Helsinki-NLP/opus-mt-mul-en"
translation_model = MarianMTModel.from_pretrained(model_name).to(device)
translation_tokenizer = MarianTokenizer.from_pretrained(model_name)

# Load GPT-Neo for creative text generation
text_generation_model_name = "EleutherAI/gpt-neo-1.3B"
text_generation_model = AutoModelForCausalLM.from_pretrained(text_generation_model_name).to(device)
text_generation_tokenizer = AutoTokenizer.from_pretrained(text_generation_model_name)

# Add padding token to GPT-Neo tokenizer if not present
if text_generation_tokenizer.pad_token is None:
    text_generation_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set your Hugging Face API key
os.environ['HF_API_KEY'] = 'Your_HF_TOKEN'  # Replace with your actual API key
api_key = os.getenv('HF_API_KEY')
if api_key is None:
    raise ValueError("Hugging Face API key is not set. Please set it in your environment.")

headers = {"Authorization": f"Bearer {api_key}"}

# Define the API URL for image generation (replace with actual model URL)
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-schnell"  # Replace with a valid image generation model

# Query Hugging Face API to generate image with error handling
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        print(f"Response: {response.text}")
        return None
    return response.content

# Translate Tamil text to English
def translate_text(tamil_text):
    inputs = translation_tokenizer(tamil_text, return_tensors="pt", padding=True, truncation=True).to(device)
    translated_tokens = translation_model.generate(**inputs)
    translation = translation_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translation

# Generate an image based on the translated text with error handling
def generate_image(prompt):
    image_bytes = query({"inputs": prompt})

    if image_bytes is None:
        # Return a blank image with error message
        error_img = Image.new('RGB', (300, 300), color=(255, 0, 0))
        d = ImageDraw.Draw(error_img)
        d.text((10, 150), "Image Generation Failed", fill=(255, 255, 255))
        return error_img

    try:
        image = Image.open(io.BytesIO(image_bytes))
        return image
    except Exception as e:
        print(f"Error: {e}")
        # Return an error image in case of failure
        error_img = Image.new('RGB', (300, 300), color=(255, 0, 0))
        d = ImageDraw.Draw(error_img)
        d.text((10, 150), "Invalid Image Data", fill=(255, 255, 255))
        return error_img

# Generate creative text based on the translated English text
def generate_creative_text(translated_text):
    inputs = text_generation_tokenizer(translated_text, return_tensors="pt", padding=True, truncation=True).to(device)
    generated_tokens = text_generation_model.generate(**inputs, max_length=100)
    creative_text = text_generation_tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return creative_text

# Function to handle the full workflow
def translate_generate_image_and_text(tamil_text):
    # Step 1: Translate Tamil to English
    translated_text = translate_text(tamil_text)

    # Step 2: Generate an image from the translated text
    image = generate_image(translated_text)

    # Step 3: Generate creative text from the translated text
    creative_text = generate_creative_text(translated_text)

    return translated_text, creative_text, image

# Create a visually appealing Gradio interface
css = """
#transart-title {
    font-size: 2.5em;
    font-weight: bold;
    color: #4CAF50;
    text-align: center;
    margin-bottom: 10px;
}
#transart-subtitle {
    font-size: 1.25em;
    text-align: center;
    color: #555555;
    margin-bottom: 20px;
}
body {
    background-color: #f0f0f5;
}
.gradio-container {
    font-family: 'Arial', sans-serif;
}
"""

# Custom HTML for title and subtitle (can be displayed in Markdown)
title_markdown = """
# <div id="transart-title">TransArt</div>
### <div id="transart-subtitle">Tamil to English Translation, Creative Text & Image Generation</div>
"""

# Gradio interface with customized layout and aesthetics
with gr.Blocks(css=css) as interface:
    gr.Markdown(title_markdown)  # Title and subtitle in Markdown
    with gr.Row():
        with gr.Column():
            tamil_input = gr.Textbox(label="Enter Tamil Text", placeholder="Type Tamil text here...", lines=3)  # Input for Tamil text
        with gr.Column():
            translated_output = gr.Textbox(label="Translated Text", interactive=False)        # Output for translated text
            creative_text_output = gr.Textbox(label="Creative Generated Text", interactive=False)  # Output for creative text
            generated_image_output = gr.Image(label="Generated Image")  # Output for generated image

    gr.Button("Generate").click(fn=translate_generate_image_and_text, inputs=tamil_input, outputs=[translated_output, creative_text_output, generated_image_output])

# Launch the Gradio app
interface.launch(debug=True, server_name="0.0.0.0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://245b986ece465b66d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://245b986ece465b66d5.gradio.live


Streamlit Code

In [ ]:
!pip install streamlit requests

In [ ]:
# Import required libraries
import streamlit as st
import requests
from getpass import getpass
from PIL import Image
import io

# Input your Hugging Face and Groq tokens securely
Transalate_token = getpass("Enter Hugging Face Translation Token: ")
Image_Token = getpass("Enter Hugging Face Image Generation Token: ")
Content_Token = getpass("Enter Groq Content Generation Token: ")
Image_prompt_token = getpass("Enter Groq Prompt Generation Token: ")

# API Headers
Translate = {"Authorization": f"Bearer {Transalate_token}"}
Image_generation = {"Authorization": f"Bearer {Image_Token}"}
Content_generation = {
    "Authorization": f"Bearer {Content_Token}",
    "Content-Type": "application/json"
}
Image_Prompt = {
    "Authorization": f"Bearer {Image_prompt_token}",
    "Content-Type": "application/json"
}

# Translation Model API URL (Tamil to English)
translation_url = "https://api-inference.huggingface.co/models/facebook/mbart-large-50-many-to-one-mmt"

# Text-to-Image Model API URLs
image_generation_urls = {
    "black-forest-labs/FLUX.1-schnell": "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-schnell",
    "CompVis/stable-diffusion-v1-4": "https://api-inference.huggingface.co/models/CompVis/stable-diffusion-v1-4",
    "black-forest-labs/FLUX.1-dev": "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
}

# Content generation models
content_models = {
    "llama-3.1-70b-versatile": "llama-3.1-70b-versatile",
    "llama3-8b-8192": "llama3-8b-8192",
    "gemma2-9b-it": "gemma2-9b-it",
    "mixtral-8x7b-32768": "mixtral-8x7b-32768"
}

# Function to query Hugging Face translation model
def translate_text(text):
    payload = {"inputs": text}
    response = requests.post(translation_url, headers=Translate, json=payload)
    if response.status_code == 200:
        result = response.json()
        translated_text = result[0]['generated_text']
        return translated_text
    else:
        return f"Translation Error {response.status_code}: {response.text}"

# Function to query Groq content generation model
def generate_content(english_text, max_tokens, temperature, model):
    url = "https://api.groq.com/openai/v1/chat/completions"
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a creative and insightful writer."},
            {"role": "user", "content": f"Write educational content about {english_text} within {max_tokens} tokens."}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature
    }
    response = requests.post(url, json=payload, headers=Content_generation)
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        return f"Content Generation Error: {response.status_code}"

# Function to generate image prompt
def generate_image_prompt(english_text):
    payload = {
        "model": "mixtral-8x7b-32768",
        "messages": [
            {"role": "system", "content": "You are a professional Text to image prompt generator."},
            {"role": "user", "content": f"Create a text to image generation prompt about {english_text} within 30 tokens."}
        ],
        "max_tokens": 30
    }
    response = requests.post("https://api.groq.com/openai/v1/chat/completions", json=payload, headers=Image_Prompt)
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        return f"Prompt Generation Error: {response.status_code}"

# Function to generate an image from the prompt
def generate_image(image_prompt, model_url):
    data = {"inputs": image_prompt}
    response = requests.post(model_url, headers=Image_generation, json=data)
    if response.status_code == 200:
        return response.content
    else:
        return f"Image Generation Error {response.status_code}: {response.text}"

# Streamlit App
def main():
    st.title("")
    st.write("Translate Tamil to English, generate educational content, and generate related images!")

    # Input Section
    tamil_input = st.text_area("Enter Tamil Text")
    temperature = st.slider("Temperature", min_value=0.1, max_value=1.0, value=0.7)
    max_tokens = st.slider("Max Tokens for Content Generation", min_value=100, max_value=400, value=200)
    content_model = st.selectbox("Select Content Generation Model", list(content_models.keys()))
    image_model = st.selectbox("Select Image Generation Model", list(image_generation_urls.keys()))

    if st.button("Generate"):
        # Step 1: Translation (Tamil to English)
        english_text = translate_text(tamil_input)

        # Step 2: Generate Educational Content
        content_output = generate_content(english_text, max_tokens, temperature, content_models[content_model])

        # Step 3: Generate Image from the prompt
        image_prompt = generate_image_prompt(english_text)
        image_data = generate_image(image_prompt, image_generation_urls[image_model])

        # Display Results
        st.subheader("Translated English Text")
        st.write(english_text)

        st.subheader("Generated Content")
        st.write(content_output)

        # Display Generated Image
        if isinstance(image_data, bytes):
            image = Image.open(io.BytesIO(image_data))
            st.image(image, caption='Generated Image', use_column_width=True)
        else:
            st.error(image_data)

# Run the app
if __name__ == "__main__":
    main()
